# Topic Modeling with Gensim

A **topic model** is an abstraction of the major topics contained in a corpus of texts. "Topic" in this context simply means a pattern of co-occurring words. The assumption is that if there are clearly identified patterns of co-occurring words, those patterns of co-occurring words reveal a latent structure in the corpus of texts. In short, a topic model is a representation of the major themes or structures of a corpus of texts.

`Gensim` is a popular Python library for building topic models. In this notebook we will use `Gensim` to build a topic model of Gibbon's _Decline and Fall of the Roman Empire_. After building a topic model, we will then use `pyLDAvis` to visualize the model so we can evaluate its usefulness.

I highly recommend that you read through `Gensim`'s [documentation](https://radimrehurek.com/gensim/auto_examples/index.html#core-tutorials-new-users-start-here). Much of the code below is adapted from that source.

## Set up

**NOTE**: one of the Python libraries we are using (`pyLDAvis`) can cause problems. Be sure to do the installations in the order that you see them below.

In [1]:
! pip install funcy

In [2]:
! pip install tzdata

In [3]:
! pip install --no-dependencies pyLDAvis

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 24.4 MB/s eta 0:00:00a 0:00:01


In [4]:
! pip install wget

In [6]:
! pip install gensim

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.0/24.0 MB 37.9 MB/s eta 0:00:0000:0100:01
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pyldavis 3.4.1 requires numexpr, which is not installed.


In [2]:
from collections import defaultdict
import wget
from gensim import corpora, models
 # <-- only necessary if you are using Colab
import pandas as pd
import pyLDAvis.gensim
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

## Upload data

### Class example

In [7]:
url = 'https://chroniclingamerica.loc.gov/search/pages/results/?state=&date1=1770&date2=1963&proxtext=Cold+War&x=0&y=0&dateFilterType=yearRange&rows=20&searchType=basic&format=json'
file_name = wget.download(url)
df = pd.read_csv(file_name)
df.head()

-1 / unknown

,"{""totalItems"": 26456","""endIndex"": 20","""startIndex"": 1","""itemsPerPage"": 20","""items"": [{""sequence"": 99","""county"": [null]","""edition"": null","""frequency"": ""Daily""","""id"": ""/lccn/sn83045462/1950-04-16/ed-1/seq-99/""","""subject"": [""Washington (D.C.)--fast--(OCoLC)fst01204505""",...,1\ny J W6L\u2014Comeras,Ist Floor \u2022\nII ... also Chevy Chase,7 Corners and Alexandria\nV L \u2713\nSHOP EARLY. SHOP LATE THURSDAYS AT W&L ash inf ton Store,open Thurt>day\u00a7. 9;3ft \u25a0jh to 9 pm. . . . Chen Uhiae. 7 Corner* and Alexandria,"9 39 h.m. to 9:39 pa.\nI\n* \u00ab""","""batch"": ""dlc_salome_ver01""","""title_normal"": ""evening star."".13","""url"": ""https://chroniclingamerica.loc.gov/lccn/sn83045462/1959-02-04/ed-1/seq-20.json""","""place"": [""District of Columbia--Washington""].13","""page"": ""A-20""}]}"


### Upload your own data sets

If you are using Google Colab:

In [ ]:
# uploaded = files.upload()

In [ ]:
#file_name = 'result.csv'# <-- for example: 'data.csv'
#df = pd.read_csv(io.BytesIO(uploaded[file_name]))

If you are using Jupyter Notebooks:

In [13]:
path = '/result.csv' # <-- for example: 'path/to/file/'
file_name = 'World_War_II_1918-1963.csv' # <-- for example: 'data.csv'
df = pd.read_csv(file_name)

## Prepare data for topic model
The Python library we are going to use to make our topic model requires the data to be in a form of a list. Within that list, each "document" is also a list. So it looks something like this:

`[
  ['This is document 1'],
  ['This is document 2'],
  ['This is document 3']
]`

In [14]:
# extract the data out of the DataFrame
documents = df['lemmas'].to_list()
print(documents)

['classification baseball non essential industry work fight order bring sharp protest tener sun monday june baseball essential war period k n morale public sa toner protests jen crow tier order ft learnt hkaoh maji june john tenor president na tional lonirue lernoon announce lie iii protest propose flsiflcatloit ot baseball industry oen crovvdcr nerk light order trround h necessity nation ueepinir period world war nddltlon v protest tho crpund baseball million dol rs invest action tend away rational pamlme work havoc lnestnient involve tener assert oen crow professional base mil n non essential industry unjii t discrimination inrnltist pinlzatlon dolus pre text ii s pt pit public tlirlnr tvso trj iiik lime national jame clean branch athletic considerable burden nf public llshter well fit topic war duty appear statement follow e time will thlrc possible win war arc mr eed point believe rlsln proper baseball player hould iepond call come treni class selective rt aft place local board cn 

In [15]:
len(documents)
len(documents[0])

27264

`Gensim` needs each document to be tokenized. We can use [list comprehension](https://www.w3schools.com/python/python_lists_comprehension.asp) to quickly achieve this result. When complete, our data will now look like this:

`[
  ['This', 'is', 'document', '1'],
  ['This', 'is', 'document', '2'],
  ['This', 'is', 'document', '3'],
]`

In [31]:
# tokenize - the syntax below will create a list of lists
texts =[
    [word for word in document.lower().split()]
    for document in documents
]

It takes a lot of preparation to build a useful topic model. An important part of that preparation is to eliminate "noise" from you model. One way to do this is to remove pieces of data that are irrelevant. Here we will remove tokens that only occur once. **You may want to adjust this as you refine your topic model.**

In [32]:
# create a count of each token
frequency = defaultdict(int)
for text in texts:
  for token in text:
    frequency[token] += 1

In [33]:
# remove words that appear only 1 time
texts = [
    [token for token in text if frequency[token] > 1]
    for text in texts
]

## Build topic model

`Gensim` is built around [four core concepts](https://radimrehurek.com/gensim/auto_examples/core/run_core_concepts.html#core-concepts):
- **document**: an individual text. In our case, this is an individual section from Gibbon.
- **corpus**: a collection of documents. In our case, this is all the sections from Gibbon put together.
- **vector**: a mathematically convenient representation of a document. Basically, each word in the document is given a numerical id. This allows `Gensim` to do faster calculations behind the scene.
- **model**: an algorithm for transforming vectors from one representation to another. In our case, this will be the LDA model we build.


### Basic topic model



In [34]:
# create a dictionary based off our texts
# The dictionary maps each token to a unique integer id
dictionary = corpora.Dictionary(texts)

In [35]:
# create a corpus based off our dictionary and our texts
corpus = [dictionary.doc2bow(text) for text in texts]

In [36]:
# build LDA model
lda_model = models.LdaModel(corpus=corpus, id2word=dictionary, num_topics=20, passes=50)

In [37]:
# explore topics
lda_model.print_topics()

[(0,
  '0.001*"tho" + 0.001*"ft" + 0.001*"n" + 0.001*"ii" + 0.001*"r" + 0.001*"s" + 0.001*"york" + 0.001*"new" + 0.001*"win" + 0.001*"t"'),
 (1,
  '0.001*"tho" + 0.001*"ft" + 0.001*"n" + 0.001*"t" + 0.001*"day" + 0.001*"new" + 0.001*"ii" + 0.001*"york" + 0.001*"r" + 0.001*"o"'),
 (2,
  '0.001*"tho" + 0.001*"n" + 0.001*"ft" + 0.001*"day" + 0.001*"new" + 0.001*"t" + 0.001*"year" + 0.001*"man" + 0.001*"ii" + 0.001*"s"'),
 (3,
  '0.026*"enmity" + 0.013*"lll" + 0.001*"new" + 0.001*"race" + 0.001*"york" + 0.001*"s" + 0.001*"good" + 0.001*"close" + 0.001*"j" + 0.001*"time"'),
 (4,
  '0.001*"tho" + 0.001*"n" + 0.001*"new" + 0.001*"day" + 0.001*"ft" + 0.001*"time" + 0.001*"ii" + 0.001*"year" + 0.001*"york" + 0.001*"r"'),
 (5,
  '0.001*"tho" + 0.001*"ft" + 0.001*"n" + 0.001*"t" + 0.001*"ii" + 0.001*"o" + 0.001*"new" + 0.001*"york" + 0.001*"r" + 0.001*"year"'),
 (6,
  '0.001*"tho" + 0.001*"n" + 0.001*"new" + 0.001*"day" + 0.001*"ii" + 0.001*"s" + 0.001*"ft" + 0.001*"t" + 0.001*"l" + 0.001*"time"'

In [38]:
# Find topics in each document
lda_model.get_document_topics(corpus[0])

[(13, 0.8849228), (15, 0.11478521)]

In [39]:
# visualize
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, dictionary)
vis

TypeError: Object of type complex is not JSON serializable

PreparedData(topic_coordinates=                        x                   y  topics  cluster       Freq
topic                                                                    
15    -0.061050+0.000000j  0.114156+0.000000j       1        1  65.482475
13     0.264561+0.000000j  0.020270+0.000000j       2        1  34.445964
3     -0.017400+0.000000j -0.020932+0.000000j       3        1   0.039362
0     -0.010947+0.000000j -0.006676+0.000000j       4        1   0.001894
11    -0.010948+0.000000j -0.006677+0.000000j       5        1   0.001894
18    -0.010947+0.000000j -0.006677+0.000000j       6        1   0.001894
17    -0.010948+0.000000j -0.006676+0.000000j       7        1   0.001894
16    -0.010948+0.000000j -0.006677+0.000000j       8        1   0.001894
14    -0.010948+0.000000j -0.006676+0.000000j       9        1   0.001894
12    -0.010949+0.000000j -0.006677+0.000000j      10        1   0.001894
10    -0.010947+0.000000j -0.006675+0.000000j      11        1   0.001894
1     -

### Tf-idf topic model

In [25]:
# initialize a tfidf model
tfidf = models.TfidfModel(corpus)

In [26]:
# make a new corpus based on the tfidf model
corpus_tfidf = tfidf[corpus]

In [27]:
# here we build our topic model
lda_model_tfidf = models.LdaModel(corpus_tfidf, id2word=dictionary, num_topics=20, passes=50)
corpus_lda = lda_model_tfidf[corpus_tfidf]

In [28]:
lda_model_tfidf.print_topics()

[(0,
  '0.001*"dry" + 0.001*"doubt" + 0.001*"electric" + 0.001*"editor" + 0.001*"edith" + 0.001*"east" + 0.001*"early" + 0.001*"eagle" + 0.001*"dunne" + 0.001*"dulck"'),
 (1,
  '0.001*"dry" + 0.001*"doubt" + 0.001*"electric" + 0.001*"editor" + 0.001*"edith" + 0.001*"east" + 0.001*"early" + 0.001*"eagle" + 0.001*"dunne" + 0.001*"dulck"'),
 (2,
  '0.001*"dry" + 0.001*"doubt" + 0.001*"electric" + 0.001*"editor" + 0.001*"edith" + 0.001*"east" + 0.001*"early" + 0.001*"eagle" + 0.001*"dunne" + 0.001*"dulck"'),
 (3,
  '0.005*"sho" + 0.004*"ford" + 0.003*"woman" + 0.003*"harvard" + 0.003*"jury" + 0.003*"wero" + 0.003*"revolver" + 0.003*"havo" + 0.002*"strieker" + 0.002*"shutz"'),
 (4,
  '0.001*"dry" + 0.001*"doubt" + 0.001*"electric" + 0.001*"editor" + 0.001*"edith" + 0.001*"east" + 0.001*"early" + 0.001*"eagle" + 0.001*"dunne" + 0.001*"dulck"'),
 (5,
  '0.005*"league" + 0.004*"ball" + 0.003*"giants" + 0.003*"baseball" + 0.003*"tennis" + 0.003*"june" + 0.003*"oo" + 0.003*"base" + 0.002*"hit" +

In [29]:
# Find topics in each document
lda_model_tfidf.get_document_topics(corpus_tfidf[0])

[(5, 0.82245713)]

In [30]:
# visualize
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model_tfidf, corpus_tfidf, dictionary)
vis

TypeError: Object of type complex is not JSON serializable

PreparedData(topic_coordinates=                        x                   y  topics  cluster       Freq
topic                                                                    
9      0.002047+0.000000j -0.008394+0.000000j       1        1  32.757299
5      0.007284+0.000000j  0.005993+0.000000j       2        1  26.673297
3     -0.009605+0.000000j  0.002755+0.000000j       3        1  25.114022
0      0.000016+0.000000j -0.000021+0.000000j       4        1   0.909140
12     0.000016+0.000000j -0.000021+0.000000j       5        1   0.909140
18     0.000016+0.000000j -0.000021+0.000000j       6        1   0.909140
17     0.000016+0.000000j -0.000021+0.000000j       7        1   0.909140
16     0.000016+0.000000j -0.000021+0.000000j       8        1   0.909140
15     0.000016+0.000000j -0.000021+0.000000j       9        1   0.909140
14     0.000016+0.000000j -0.000021+0.000000j      10        1   0.909140
13     0.000016+0.000000j -0.000021+0.000000j      11        1   0.909140
10     